In [1]:
from __future__ import print_function
from random import randint
from tensorflow import keras
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation
import tensorflow as tf
import time
import math

In [2]:
batch_size = 256
num_classes = 10
epochs = 20

In [3]:
# train the model
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print(Y_train.shape)
print(Y_test.shape)

(60000,)
(10000,)


In [4]:
def normalize(array,quanti=8):#normalize and quantize the weight, denormalize in feedfoward
    w=np.asarray(array)
    absmax=np.amax(np.abs(w))
    w = w/absmax
    quantScale=1/quanti/2
    for i in range(-quanti,quanti):
        j=i/quanti+quantScale
        w[np.abs(w-j)<=quantScale] = j
    return w

def ifp_normalize(array,levels=16):#normalize and quantize the weight, denormalize in feedfoward
    w=np.asarray(array)
    absmax=np.amax(np.abs(w))
    w = w/absmax
    quantScale=1/levels/2
    for i in range(0,levels):
        j=i/levels+quantScale
        w[np.abs(w-j)<=quantScale] = j
    return w

In [5]:
x_train = X_train.reshape(60000, 784)
x_test = X_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train=ifp_normalize(x_train,16)
x_test=ifp_normalize(x_test,16)
num_classes = 10
y_train = tf.keras.utils.to_categorical(Y_train, num_classes)
y_test = tf.keras.utils.to_categorical(Y_test, num_classes)

In [6]:
print(y_train.shape)
print(y_test.shape)

(60000, 10)
(10000, 10)


In [8]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import ReLU
from tensorflow.keras.constraints import Constraint
from tensorflow.keras import backend as K

class WeightClip(Constraint):
    '''Clips the weights incident to each hidden unit to be inside a range
    '''
    def __init__(self, c=1):
        self.c = c

    def __call__(self, p):
        return K.clip(p, -self.c, self.c)

    def get_config(self):
        return {'name': self.__class__.__name__,
                'c': self.c}
layers=4
def propagation_loss(length=10*10e-4,loss_per_length=0.3):
    return 1/10**(length*loss_per_length/10)
def splitter_loss(loss=0.2):
    return 1/10**(loss/10)
def coupling_loss(layers=1):
    return 1/10**(layers*0.5/10)
def layer_loss(leaf=64):
    s_n = math.log2(leaf)+3
    p_n = s_n+1
    c_n = 2
    return 255/256 * 1/leaf *1/2 * (splitter_loss())**s_n * propagation_loss()**p_n *coupling_loss()**c_n
use_bias = False
def create_model():
    minPDin=2e-7
    wq_lv=7
    model = Sequential(name='model1')
    retrain= False
    global layers
    global use_bias
    if retrain: 
        for i in range(layers-1):
            model.add(Dense(256, input_dim=784,activation=None, use_bias = use_bias, kernel_constraint = WeightClip(1)))
            model.add(ReLU(max_value=1/layer_loss(), negative_slope=0.0, threshold=minPDin/layer_loss()))
        model.add(Dense(10, activation='softmax',use_bias = use_bias, kernel_constraint = WeightClip(1)))
    #model.add(GaussianNoise(noise_para))
    else:
        for i in range(layers-1):
            model.add(Dense(256,input_dim=784,use_bias = use_bias, activation='relu'))
        model.add(Dense(10,use_bias = use_bias,activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', 
                  optimizer=tf.keras.optimizers.Adam(), 
                  metrics=['accuracy'])
    return model
   

model = create_model()

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               200704    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65536     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65536     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2560      
Total params: 334,336
Trainable params: 334,336
Non-trainable params: 0
_________________________________________________________________


In [371]:
# history = model.fit(x_train, y_train,
#                     batch_size=batch_size,
#                     epochs=epochs,
#                     verbose=1,
#                     validation_data=(x_test, y_test))
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 2s 26us/sample - loss: 0.3209 - acc: 0.9059 - val_loss: 0.1470 - val_acc: 0.9565
Epoch 2/20
60000/60000 [==============================] - 1s 17us/sample - loss: 0.1148 - acc: 0.9652 - val_loss: 0.1136 - val_acc: 0.9645
Epoch 3/20
60000/60000 [==============================] - 1s 18us/sample - loss: 0.0750 - acc: 0.9769 - val_loss: 0.0860 - val_acc: 0.9718
Epoch 4/20
60000/60000 [==============================] - 1s 17us/sample - loss: 0.0531 - acc: 0.9833 - val_loss: 0.0795 - val_acc: 0.9742
Epoch 5/20
60000/60000 [==============================] - 1s 18us/sample - loss: 0.0393 - acc: 0.9874 - val_loss: 0.0728 - val_acc: 0.9771
Epoch 6/20
60000/60000 [==============================] - 1s 17us/sample - loss: 0.0332 - acc: 0.9894 - val_loss: 0.0691 - val_acc: 0.9783
Epoch 7/20
60000/60000 [==============================] - 1s 17us/sample - loss: 0.0271 - acc: 0.9910 - val_loss: 0.

In [372]:
#model.save_weights('my_model_weights.h5')

In [9]:
model2 = Sequential()
for i in range(layers-1):
    model2.add(Dense(256,input_dim=784,use_bias = use_bias, activation='relu'))
model2.add(Dense(10, use_bias = use_bias,activation='softmax'))

model2.load_weights('mnist_DACTL_weights.h5')
for i in range(0,layers):
    print(model2.layers[i].get_weights()[0].shape)

(784, 256)
(256, 256)
(256, 256)
(256, 10)


In [10]:

def feedforward2(layer,inputs,weights,bias,variation,activate_relu=1,leaf=64,spread_branch=1/256):
    # Convert signal from electrical domain to optical domain by TL in former layer\
    inputs[inputs>1]=1
    Splitter_Variation=0.05
    tl_variation=0.05
    inputs = inputs * np.random.uniform(1-tl_variation,1+tl_variation,inputs.shape) * coupling_loss() 
    copy_num = weights.shape[1]
    spliter_count = math.log2(leaf) + 3
    #print(inputs.shape[0])
    # Distribute inputs by splitter trees
    random_input = np.repeat(inputs,copy_num,0).reshape(inputs.shape[0],copy_num) \
    * spliter_forest(inputs.shape[0],copy_num,leaf,Splitter_Variation,variation,spread_branch)
    # Add variation to splitters for weight calculation 
    weights_random = weights * np.random.uniform(1-Splitter_Variation,1+Splitter_Variation,weights.shape)
    
    # Do the weight calculation 
    result = random_input * weights_random * propagation_loss() * splitter_loss()
    
    # Convert signal from optical domain to electrical domain by PD with 1:1 splitter for PD selection
    result = result * 0.5 * splitter_loss()* propagation_loss()\
    *np.random.uniform(1-Splitter_Variation,1+Splitter_Variation,result.shape) * coupling_loss() 
#     if activate_relu!=1:
#         print(np.amax(result))
    # Set lower bound for input of PD
    
    result[np.abs(result)<2e-7]=0
    
    # Accumulation
    result = np.sum (result,0)
    # Amplify the result with corresponding coeffient to obtaint correct output of this layer
  
    # If it is not the last layer, implement ReLU and set upper bound for output at 1mW
    global use_bias
    if use_bias!=False:
        for i in range(0,layer+1):
            bias =  bias * 255/256 * 1/leaf *1/2 *\
                    splitter_loss() ** spliter_count * propagation_loss() ** spliter_count * coupling_loss() ** 2
    #     print('o2 bias:',end='')
    #     print(bias)
    #     global bias_coe
    #     bias *= bias_coe
    output = result  + bias
    global layers
    if layer!=layers-1:
        output=relu(output)
#     print('o2 output:',end='')
    #print(output)
    return output


def relu(output):
  nextinput = np.zeros((output.shape[0]))
  for i in range(nextinput.shape[0]):
    nextinput[i] = max(0,output[i])
  return nextinput
#@jit

In [11]:

def network(x_test,y_test,variation):#x_test is 1*784, one 28*28 image
    weights=[]
    bias=[]
    wmax=np.zeros(layers)
    global use_bias
    for i in range(0,layers):
        weights.append(np.array(model2.layers[i].get_weights()[0]))
        if use_bias == False:
            bias.append(0)
        else:
            bias.append(np.array(model2.layers[i].get_weights()[1]))
        wmax[i]=np.amax(np.abs(weights[i]))
    o2 = feedforward2(0,x_test,normalize(weights[0]),bias[0],wmax[0],variation)
    o2 = amplifier(o2,1/layer_loss()*wmax[0],0.05)
    o2 = feedforward2(1,o2,normalize(weights[1]),bias[1],wmax[1],variation)
    o2 = amplifier(o2,1/layer_loss()*wmax[1],0.05)
    o2 = feedforward2(2,o2,normalize(weights[2]),bias[2],wmax[2],variation)
    o2 = amplifier(o2,1/layer_loss()*wmax[2],0.05)
    o2 = feedforward2(3,o2,normalize(weights[3]),bias[3],wmax[3],variation)
    pred_out = np.zeros((1,o2.shape[0]))
    pred_out[0] = o2
    correct_out = np.zeros((1,y_test.shape[0]))
    correct_out[0] = y_test
    #print(np.amax(pred_out))
    return (np.argmax(pred_out) == np.argmax(correct_out))


def spliter_tree(root_value=1,leaf=64,variation=0.05):
    height=math.ceil(math.log2(leaf))+1
    arr=np.ones(int('1'*height, 2))
    for i in range(1,height):
        left = np.random.uniform(1-variation,1+variation,2**(i-1))*1/2
        right = 1-left
        branch = np.concatenate((left,right)).flatten('F')*propagation_loss()*splitter_loss()
        arr[2**i-1:2**(i+1)-1]=np.repeat(arr[2**(i-1)-1:2**i-1],2)*branch
    return arr[2**(height-1)-1:2**height-1]
def spliter_grove(total,spliter_variation,amplifier_variation,leaf=64,spread_branch=1/256):
    tl_variation = 0.05
    tree_num=math.ceil(total/leaf)
    arr=np.zeros((tree_num,leaf))
    v=np.random.uniform(1-spliter_variation,1+spliter_variation,tree_num)
    right=np.zeros(tree_num)
    left=np.zeros(tree_num)
    right[0]=v[0]*spread_branch
    left[0]=1-right[0]
    arr[0]=spliter_tree(left[0]*propagation_loss()*splitter_loss(),leaf,spliter_variation)
    for i in range(1,tree_num):
        right[i]=amplifier(right[i-1]*splitter_loss()*propagation_loss()*coupling_loss(),\
                           1/(spread_branch*splitter_loss()*propagation_loss()*coupling_loss()),\
                           amplifier_variation)*v[i]*spread_branch
        left[i]=1-right[i]
        arr[i]=spliter_tree(left[i]*propagation_loss()*splitter_loss(),leaf,spliter_variation)
    return arr.flatten()[:total] 

def spliter_forest(input_num=784,grove_len=256,leaf=64,spliter_variation=0.05,amplifier_variation=0.05,spread_branch=1/256):
    arr=np.zeros((input_num,grove_len))
    for i in range(0,input_num):
        arr[i]=spliter_grove(grove_len,spliter_variation,amplifier_variation,leaf,spread_branch)
    return arr 

def amplifier(x, multiple, amp_vari=0.05):
    delta_in= x * np.random.uniform(-amp_vari,amp_vari)
    x_out = x * multiple + multiple * 6.25 * delta_in*10**(1/2)
    return x_out

In [12]:
print(layer_loss(64))

0.004055928080193953


In [ ]:

def main(logname='MNIST_original.txt'):
    log=open(logname,'w')
    log.write(time.strftime("%Y-%m-%d %H:%M:%S\n", time.localtime()))
    log.close()
    result=0
    j=5
    iteration = 100
    for k in range(iteration):
        correct_test = 0
        for i in range(0,x_test.shape[0]):
#             if True:
            if (network(x_test[i],y_test[i],0.01*j)):
                correct_test += 1
            if (i+1)%10==0: # to test the function
               print ('inference with variation %f current accuracy at %i:'%(0.01*j,i+1),correct_test/(i+1))
        log = open(logname,'a')
        log.write('The accuracy of inference %d is %f\n'%(k,correct_test/10000))
        result += correct_test/10000
    log=open(logname,'a')
    log.write('The average accuracy is %f\n'%(result/iteration))
    log.close()
    
main('MNISTs.txt')

inference with variation 0.050000 current accuracy at 10: 1.0
inference with variation 0.050000 current accuracy at 20: 1.0
inference with variation 0.050000 current accuracy at 30: 1.0
inference with variation 0.050000 current accuracy at 40: 1.0
inference with variation 0.050000 current accuracy at 50: 1.0
inference with variation 0.050000 current accuracy at 60: 1.0
inference with variation 0.050000 current accuracy at 70: 1.0
inference with variation 0.050000 current accuracy at 80: 1.0
inference with variation 0.050000 current accuracy at 90: 0.9888888888888889
inference with variation 0.050000 current accuracy at 100: 0.99
inference with variation 0.050000 current accuracy at 110: 0.990909090909091
inference with variation 0.050000 current accuracy at 120: 0.975
inference with variation 0.050000 current accuracy at 130: 0.9692307692307692
inference with variation 0.050000 current accuracy at 140: 0.9714285714285714
inference with variation 0.050000 current accuracy at 150: 0.9666

inference with variation 0.050000 current accuracy at 1100: 0.9654545454545455
inference with variation 0.050000 current accuracy at 1110: 0.9648648648648649
inference with variation 0.050000 current accuracy at 1120: 0.9633928571428572
inference with variation 0.050000 current accuracy at 1130: 0.9628318584070796
inference with variation 0.050000 current accuracy at 1140: 0.9631578947368421
inference with variation 0.050000 current accuracy at 1150: 0.9634782608695652
inference with variation 0.050000 current accuracy at 1160: 0.9629310344827586
inference with variation 0.050000 current accuracy at 1170: 0.9615384615384616
inference with variation 0.050000 current accuracy at 1180: 0.9610169491525423
inference with variation 0.050000 current accuracy at 1190: 0.9596638655462185
inference with variation 0.050000 current accuracy at 1200: 0.9591666666666666
inference with variation 0.050000 current accuracy at 1210: 0.9586776859504132
inference with variation 0.050000 current accuracy a

inference with variation 0.050000 current accuracy at 2150: 0.9548837209302325
inference with variation 0.050000 current accuracy at 2160: 0.9550925925925926
inference with variation 0.050000 current accuracy at 2170: 0.9548387096774194
inference with variation 0.050000 current accuracy at 2180: 0.955045871559633
inference with variation 0.050000 current accuracy at 2190: 0.9538812785388128
inference with variation 0.050000 current accuracy at 2200: 0.9540909090909091
inference with variation 0.050000 current accuracy at 2210: 0.9542986425339367
inference with variation 0.050000 current accuracy at 2220: 0.9545045045045045
inference with variation 0.050000 current accuracy at 2230: 0.9542600896860987
inference with variation 0.050000 current accuracy at 2240: 0.9544642857142858
inference with variation 0.050000 current accuracy at 2250: 0.9546666666666667
inference with variation 0.050000 current accuracy at 2260: 0.9548672566371681
inference with variation 0.050000 current accuracy at

inference with variation 0.050000 current accuracy at 3200: 0.95875
inference with variation 0.050000 current accuracy at 3210: 0.9588785046728971
inference with variation 0.050000 current accuracy at 3220: 0.9590062111801242
inference with variation 0.050000 current accuracy at 3230: 0.9588235294117647
inference with variation 0.050000 current accuracy at 3240: 0.9589506172839506
inference with variation 0.050000 current accuracy at 3250: 0.9587692307692308
inference with variation 0.050000 current accuracy at 3260: 0.9588957055214724
inference with variation 0.050000 current accuracy at 3270: 0.9590214067278288
inference with variation 0.050000 current accuracy at 3280: 0.9591463414634146
inference with variation 0.050000 current accuracy at 3290: 0.9586626139817629
inference with variation 0.050000 current accuracy at 3300: 0.9587878787878787
inference with variation 0.050000 current accuracy at 3310: 0.9589123867069487
inference with variation 0.050000 current accuracy at 3320: 0.9

inference with variation 0.050000 current accuracy at 4250: 0.9585882352941176
inference with variation 0.050000 current accuracy at 4260: 0.9586854460093897
inference with variation 0.050000 current accuracy at 4270: 0.9587822014051522
inference with variation 0.050000 current accuracy at 4280: 0.9588785046728971
inference with variation 0.050000 current accuracy at 4290: 0.9587412587412587
inference with variation 0.050000 current accuracy at 4300: 0.9588372093023256
inference with variation 0.050000 current accuracy at 4310: 0.9587006960556844
inference with variation 0.050000 current accuracy at 4320: 0.9585648148148148
inference with variation 0.050000 current accuracy at 4330: 0.9586605080831408
inference with variation 0.050000 current accuracy at 4340: 0.9587557603686636
inference with variation 0.050000 current accuracy at 4350: 0.9588505747126437
inference with variation 0.050000 current accuracy at 4360: 0.9589449541284404
inference with variation 0.050000 current accuracy a

...

inference with variation 0.050000 current accuracy at 3730: 0.9587131367292225
inference with variation 0.050000 current accuracy at 3740: 0.9588235294117647
inference with variation 0.050000 current accuracy at 3750: 0.9589333333333333
inference with variation 0.050000 current accuracy at 3760: 0.9587765957446809
inference with variation 0.050000 current accuracy at 3770: 0.9588859416445623
inference with variation 0.050000 current accuracy at 3780: 0.9584656084656085
inference with variation 0.050000 current accuracy at 3790: 0.9583113456464379
inference with variation 0.050000 current accuracy at 3800: 0.9581578947368421
inference with variation 0.050000 current accuracy at 3810: 0.958005249343832
inference with variation 0.050000 current accuracy at 3820: 0.9575916230366492
inference with variation 0.050000 current accuracy at 3830: 0.9574412532637075
inference with variation 0.050000 current accuracy at 3840: 0.9575520833333333
inference with variation 0.050000 current accuracy at

inference with variation 0.050000 current accuracy at 4780: 0.9585774058577405
inference with variation 0.050000 current accuracy at 4790: 0.9586638830897704
inference with variation 0.050000 current accuracy at 4800: 0.9585416666666666
inference with variation 0.050000 current accuracy at 4810: 0.9584199584199584
inference with variation 0.050000 current accuracy at 4820: 0.958298755186722
inference with variation 0.050000 current accuracy at 4830: 0.9581780538302277
inference with variation 0.050000 current accuracy at 4840: 0.9580578512396695
inference with variation 0.050000 current accuracy at 4850: 0.9581443298969072
inference with variation 0.050000 current accuracy at 4860: 0.9582304526748971
inference with variation 0.050000 current accuracy at 4870: 0.9581108829568789
inference with variation 0.050000 current accuracy at 4880: 0.9581967213114754
inference with variation 0.050000 current accuracy at 4890: 0.9580777096114519
inference with variation 0.050000 current accuracy at

inference with variation 0.050000 current accuracy at 5830: 0.9624356775300171
inference with variation 0.050000 current accuracy at 5840: 0.9625
inference with variation 0.050000 current accuracy at 5850: 0.9625641025641025
inference with variation 0.050000 current accuracy at 5860: 0.9626279863481229
inference with variation 0.050000 current accuracy at 5870: 0.9626916524701874
inference with variation 0.050000 current accuracy at 5880: 0.9627551020408164
inference with variation 0.050000 current accuracy at 5890: 0.9628183361629881
inference with variation 0.050000 current accuracy at 5900: 0.9627118644067797
inference with variation 0.050000 current accuracy at 5910: 0.9627749576988156
inference with variation 0.050000 current accuracy at 5920: 0.9628378378378378
inference with variation 0.050000 current accuracy at 5930: 0.9629005059021922
inference with variation 0.050000 current accuracy at 5940: 0.9626262626262626
inference with variation 0.050000 current accuracy at 5950: 0.96

inference with variation 0.050000 current accuracy at 6880: 0.9636627906976745
inference with variation 0.050000 current accuracy at 6890: 0.9637155297532656
inference with variation 0.050000 current accuracy at 6900: 0.9637681159420289
inference with variation 0.050000 current accuracy at 6910: 0.9638205499276411
inference with variation 0.050000 current accuracy at 6920: 0.9638728323699421
inference with variation 0.050000 current accuracy at 6930: 0.9637806637806637
inference with variation 0.050000 current accuracy at 6940: 0.9638328530259366
inference with variation 0.050000 current accuracy at 6950: 0.9638848920863309
inference with variation 0.050000 current accuracy at 6960: 0.9639367816091954
inference with variation 0.050000 current accuracy at 6970: 0.9639885222381636
inference with variation 0.050000 current accuracy at 6980: 0.9640401146131805
inference with variation 0.050000 current accuracy at 6990: 0.9640915593705294
inference with variation 0.050000 current accuracy a

inference with variation 0.050000 current accuracy at 7920: 0.9664141414141414
inference with variation 0.050000 current accuracy at 7930: 0.9663303909205548
inference with variation 0.050000 current accuracy at 7940: 0.9662468513853905
inference with variation 0.050000 current accuracy at 7950: 0.9662893081761006
inference with variation 0.050000 current accuracy at 7960: 0.9663316582914573
inference with variation 0.050000 current accuracy at 7970: 0.9663739021329988
inference with variation 0.050000 current accuracy at 7980: 0.9664160401002506
inference with variation 0.050000 current accuracy at 7990: 0.9664580725907385
inference with variation 0.050000 current accuracy at 8000: 0.9665
inference with variation 0.050000 current accuracy at 8010: 0.966541822721598
inference with variation 0.050000 current accuracy at 8020: 0.9665835411471322
inference with variation 0.050000 current accuracy at 8030: 0.9666251556662515
inference with variation 0.050000 current accuracy at 8040: 0.966